In [1]:
import numpy as np
import pandas as pd 
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
%matplotlib inline

## 数据规整
金融问题域有关的话题。

### 时间序列以及界面对齐
在处理金融数据时，最费神的一个问题就是所谓的“数据对齐”（data alignment）问题。

In [2]:
prices = pd.read_csv('pydata-book-master/ch11/stock_px.csv', parse_dates=True, index_col=0).loc['2011-09-06': '2011-09-14', ['AAPL', 'JNJ', 'SPX', 'XOM']]

In [3]:
volume = pd.read_csv('pydata-book-master/ch11/volume.csv', parse_dates=True, index_col=0).loc['2011-09-06': '2011-09-12', ['AAPL', 'JNJ', 'XOM']]

In [4]:
prices

,AAPL,JNJ,SPX,XOM
2011-09-06,379.74,64.64,1165.24,71.15
2011-09-07,383.93,65.43,1198.62,73.65
2011-09-08,384.14,64.95,1185.90,72.82
2011-09-09,377.48,63.64,1154.23,71.01
2011-09-12,379.94,63.59,1162.27,71.84
2011-09-13,384.62,63.61,1172.87,71.65
2011-09-14,389.30,63.73,1188.68,72.64


In [5]:
volume

,AAPL,JNJ,XOM
2011-09-06,18173500.0,15848300.0,25416300.0
2011-09-07,12492000.0,10759700.0,23108400.0
2011-09-08,14839800.0,15551500.0,22434800.0
2011-09-09,20171900.0,17008200.0,27969100.0
2011-09-12,16697300.0,13448200.0,26205800.0


假设你想要用所有有效数据计算一个成交量加权平均价格（为了简单起见，假设成交量数据是价格数据的子集）。由于pandas会在算术运算过程中自动将数据对齐，并在sum这样的函数中排除缺失数据：

In [6]:
prices * volume

,AAPL,JNJ,SPX,XOM
2011-09-06,6.901205e+09,1.024434e+09,NaN,1.808370e+09
2011-09-07,4.796054e+09,7.040072e+08,NaN,1.701934e+09
2011-09-08,5.700561e+09,1.010070e+09,NaN,1.633702e+09
2011-09-09,7.614489e+09,1.082402e+09,NaN,1.986086e+09
2011-09-12,6.343972e+09,8.551710e+08,NaN,1.882625e+09
2011-09-13,NaN,NaN,NaN,NaN
2011-09-14,NaN,NaN,NaN,NaN


In [7]:
vwap = (prices * volume).sum() / volume.sum()

In [8]:
vwap

AAPL    380.655181
JNJ      64.394769
SPX            NaN
XOM      72.024288
dtype: float64

In [9]:
vwap.dropna()

AAPL    380.655181
JNJ      64.394769
XOM      72.024288
dtype: float64

由于SPX在volume中找不到，所以你随时可以显式地将其丢弃。如果希望手工进行对齐，可以使用DataFrame的align方法，它返回的是一个元组，含有两个对象的重索引版本：

In [10]:
prices.align(volume, join='inner')

(              AAPL    JNJ    XOM
 2011-09-06  379.74  64.64  71.15
 2011-09-07  383.93  65.43  73.65
 2011-09-08  384.14  64.95  72.82
 2011-09-09  377.48  63.64  71.01
 2011-09-12  379.94  63.59  71.84,
                   AAPL         JNJ         XOM
 2011-09-06  18173500.0  15848300.0  25416300.0
 2011-09-07  12492000.0  10759700.0  23108400.0
 2011-09-08  14839800.0  15551500.0  22434800.0
 2011-09-09  20171900.0  17008200.0  27969100.0
 2011-09-12  16697300.0  13448200.0  26205800.0)

另一个不可或缺的功能是，通过一组索引可能不同的Series构建一个DataFRame。

In [11]:
s1 = Series(range(3), index=['a', 'b', 'c'])

In [12]:
s2 = Series(range(4), index=['d', 'b', 'c', 'e'])

In [13]:
s3 = Series(range(3), index=['f', 'a', 'c'])

In [14]:
DataFrame({'one': s1, 'two': s2, 'three': s3})

,one,three,two
a,0.0,1.0,NaN
b,1.0,NaN,1.0
c,2.0,2.0,2.0
d,NaN,NaN,0.0
e,NaN,NaN,3.0
f,NaN,0.0,NaN


跟前面一样，这里也可以显式定义结果的索引（丢弃其余的数据）：

In [15]:
DataFrame({'one': s1, 'two': s2, 'three': s3}, index=list('face'))

,one,three,two
f,NaN,0.0,NaN
a,0.0,1.0,NaN
c,2.0,2.0,2.0
e,NaN,NaN,3.0


### 频率不同的时间序列的运算
频率转换和重对齐的两大主要工具是resample和reindex方法。

In [16]:
ts1 = Series(np.random.randn(3),
             index=pd.date_range('2012-6-13', periods=3, freq='W-WED'))

In [17]:
ts1

2012-06-13   -1.874289
2012-06-20   -1.650114
2012-06-27    1.793625
Freq: W-WED, dtype: float64

如果将其重采样到工作日（星期一到星期五）频率，则那些没有数据的日子就会出现一个空洞：

In [18]:
ts1.resample('B').mean()

2012-06-13   -1.874289
2012-06-14         NaN
2012-06-15         NaN
2012-06-18         NaN
2012-06-19         NaN
2012-06-20   -1.650114
2012-06-21         NaN
2012-06-22         NaN
2012-06-25         NaN
2012-06-26         NaN
2012-06-27    1.793625
Freq: B, dtype: float64

当然，只需将fill_method设置为'ffill'即可用前面的值填充这些空白。处理较低频率的数据时常这么干，因为最终结果中各时间点都有一个最新的有效值：

In [19]:
ts1.resample('B').ffill()

2012-06-13   -1.874289
2012-06-14   -1.874289
2012-06-15   -1.874289
2012-06-18   -1.874289
2012-06-19   -1.874289
2012-06-20   -1.650114
2012-06-21   -1.650114
2012-06-22   -1.650114
2012-06-25   -1.650114
2012-06-26   -1.650114
2012-06-27    1.793625
Freq: B, dtype: float64

在实际工作中，将较低频率的数据升采样到较高的规整频率是一种不错的解决方案，但是对于更一般化的不规整时间序列可能就不太合适了。

In [20]:
dates = pd.DatetimeIndex(['2016-6-12', '2012-6-17', '2012-6-18',
                          '2012-6-21', '2012-6-22', '2012-6-29'])

In [21]:
ts2 = Series(np.random.randn(6), index=dates)

In [22]:
ts2

2016-06-12   -0.016062
2012-06-17    0.183422
2012-06-18   -0.357692
2012-06-21   -0.665632
2012-06-22   -1.356151
2012-06-29   -1.588390
dtype: float64

如果要将是ts1中“最当前”值（即前向填充）加到ts2上。一个办法是将两者重采样为规整频率后再相加，但是如果想维持ts2中的日期索引，则reindex会是一种更好的解决方案：

In [23]:
ts1.reindex(ts2.index, method='ffill')

2016-06-12    1.793625
2012-06-17   -1.874289
2012-06-18   -1.874289
2012-06-21   -1.650114
2012-06-22   -1.650114
2012-06-29    1.793625
dtype: float64

In [24]:
ts2 + ts1.reindex(ts2.index, method='ffill')

2016-06-12    1.777563
2012-06-17   -1.690867
2012-06-18   -2.231982
2012-06-21   -2.315746
2012-06-22   -3.006265
2012-06-29    0.205235
dtype: float64

### 使用Period
Period（表示时间区间）提供了另一种处理不同频率时间序列的方法，尤其是那些有着特殊规范的以年或季度为频率的金融或经济序列。比如说，一个公司可能会发布其以6月份结尾的财年的每季度盈利报告，即频率为Q-JUN。来看两个有关GDP和通货膨胀的宏观经济时间序列：

In [25]:
gdp = Series([1.78, 1.94, 2.08, 2.01, 2.15, 2.31, 2.46],
             index=pd.period_range('1984Q2', periods=7, freq='Q-SEP'))

In [26]:
infl = Series([0.025, 0.045, 0.037, 0.04],
              index=pd.period_range('1982', periods=4, freq='A-DEC'))

In [27]:
gdp

1984Q2    1.78
1984Q3    1.94
1984Q4    2.08
1985Q1    2.01
1985Q2    2.15
1985Q3    2.31
1985Q4    2.46
Freq: Q-SEP, dtype: float64

In [28]:
infl

1982    0.025
1983    0.045
1984    0.037
1985    0.040
Freq: A-DEC, dtype: float64

跟Timestamp的时间序列不同，由Period索引的两个不同频率的时间序列之间的运算必须进行显式转换。在本例中，假设已知infl值是在每年年末观测的，于是我们就可以将其转换到Q-SEP以得到该频率下的正确时期：

In [29]:
infl_q = infl.asfreq('Q-SEP', how='end')

In [30]:
infl_q

1983Q1    0.025
1984Q1    0.045
1985Q1    0.037
1986Q1    0.040
Freq: Q-SEP, dtype: float64

然后这个时间序列就可以被重索引了（使用前向填充以匹配gdp）：

In [31]:
infl_q.reindex(gdp.index, method='ffill')

1984Q2    0.045
1984Q3    0.045
1984Q4    0.045
1985Q1    0.037
1985Q2    0.037
1985Q3    0.037
1985Q4    0.037
Freq: Q-SEP, dtype: float64

## 时间和“最当前”数据选取
假设你有一个很长的盘中市场数据时间序列，现在希望抽取其中每天特定时间的价格数据。如果数据不规整（观测值没有精确地落在期望的时间点上），该怎么办？

In [32]:
# 生成一个交易日内的日期范围和时间序列
rng = pd.date_range('2012-06-01 09:30', '2012-06-01 15:59', freq='T')

In [33]:
# 生成5天的时间点（9：30~15：59之间的值）
rng = rng.append([rng + pd.offsets.BDay(i) for i in range(1, 4)])

In [34]:
ts = Series(np.arange(len(rng), dtype=float), index=rng)

In [35]:
ts

2012-06-01 09:30:00       0.0
2012-06-01 09:31:00       1.0
2012-06-01 09:32:00       2.0
2012-06-01 09:33:00       3.0
2012-06-01 09:34:00       4.0
2012-06-01 09:35:00       5.0
2012-06-01 09:36:00       6.0
2012-06-01 09:37:00       7.0
2012-06-01 09:38:00       8.0
2012-06-01 09:39:00       9.0
2012-06-01 09:40:00      10.0
2012-06-01 09:41:00      11.0
2012-06-01 09:42:00      12.0
2012-06-01 09:43:00      13.0
2012-06-01 09:44:00      14.0
2012-06-01 09:45:00      15.0
2012-06-01 09:46:00      16.0
2012-06-01 09:47:00      17.0
2012-06-01 09:48:00      18.0
2012-06-01 09:49:00      19.0
2012-06-01 09:50:00      20.0
2012-06-01 09:51:00      21.0
2012-06-01 09:52:00      22.0
2012-06-01 09:53:00      23.0
2012-06-01 09:54:00      24.0
2012-06-01 09:55:00      25.0
2012-06-01 09:56:00      26.0
2012-06-01 09:57:00      27.0
2012-06-01 09:58:00      28.0
2012-06-01 09:59:00      29.0
                        ...  
2012-06-06 15:30:00    1530.0
2012-06-06 15:31:00    1531.0
2012-06-06

利用Python的datetime.time对象进行索引即可抽取这些时间点上的值：

In [36]:
from datetime import time

In [37]:
ts[time(10, 0)]

2012-06-01 10:00:00      30.0
2012-06-04 10:00:00     420.0
2012-06-05 10:00:00     810.0
2012-06-06 10:00:00    1200.0
dtype: float64

实际上，该操作用到了实例方法at_time（各时间序列以及类似的DataFrame对象都有）：

In [38]:
ts.at_time(time(10, 0))

2012-06-01 10:00:00      30.0
2012-06-04 10:00:00     420.0
2012-06-05 10:00:00     810.0
2012-06-06 10:00:00    1200.0
dtype: float64

还有一个between_time方法，它用于选取两个Time对象之间的值：

In [39]:
ts.between_time(time(10, 0), time(10, 1))

2012-06-01 10:00:00      30.0
2012-06-01 10:01:00      31.0
2012-06-04 10:00:00     420.0
2012-06-04 10:01:00     421.0
2012-06-05 10:00:00     810.0
2012-06-05 10:01:00     811.0
2012-06-06 10:00:00    1200.0
2012-06-06 10:01:00    1201.0
dtype: float64

正如之前提到的那样，可能刚好就没有任何数据落在某个具体的时间上（比如上午10点）。这时你可能会希望得到上午10点之前最后出现的那个值：

In [40]:
indexer = np.sort(np.random.permutation(len(ts))[700:])

In [41]:
irr_ts = ts.copy()

In [42]:
irr_ts[indexer] = np.nan

In [43]:
irr_ts['2012-06-01 09:50':'2012-06-01 10:00']

2012-06-01 09:50:00    20.0
2012-06-01 09:51:00     NaN
2012-06-01 09:52:00    22.0
2012-06-01 09:53:00     NaN
2012-06-01 09:54:00    24.0
2012-06-01 09:55:00     NaN
2012-06-01 09:56:00     NaN
2012-06-01 09:57:00     NaN
2012-06-01 09:58:00     NaN
2012-06-01 09:59:00     NaN
2012-06-01 10:00:00    30.0
dtype: float64

如果将一组Timestamp传入asof方法，就能得到这些时间点处（或其之前最近）的有效值（非NA）。例如，我们构造一个日期范围（每天上午10点），然后将其传入asof：

In [44]:
selection = pd.date_range('2012-06-01 10:00', periods=4, freq='B')

In [45]:
irr_ts.asof(selection)

2012-06-01 10:00:00      30.0
2012-06-04 10:00:00     419.0
2012-06-05 10:00:00     808.0
2012-06-06 10:00:00    1199.0
Freq: B, dtype: float64

In [46]:
irr_ts[time(10, 0)]

2012-06-01 10:00:00    30.0
2012-06-04 10:00:00     NaN
2012-06-05 10:00:00     NaN
2012-06-06 10:00:00     NaN
dtype: float64

## 拼接多个数据源
在金融领域中，经常出现的情况：
* 在一个特定的时间点上，从一个数据源切换到另一个数据源。
* 用另一个时间序列对当前时间序列中的缺失值“打补丁”。
* 将数据中的符号（国家、资产代码等）替换为实际数据。

对于第一种情况，在特定时刻从一个时间序列切换到另一个，其实就是用pandas.concat将两个TimeSeries或DataFrame对象合并到一起：

In [47]:
data1 = DataFrame(np.ones((6, 3), dtype=float),
                  columns=['a', 'b', 'c'],
                  index=pd.date_range('6/12/2012', periods=6))

In [48]:
data2 = DataFrame(np.ones((6, 3), dtype=float) * 2,
                  columns=['a', 'b', 'c'],
                  index=pd.date_range('6/13/2012', periods=6))

In [49]:
spliced = pd.concat([data1.loc[:'2012-06-14'], data2.loc['2012-06-15':]])

In [50]:
spliced

,a,b,c
2012-06-12,1.0,1.0,1.0
2012-06-13,1.0,1.0,1.0
2012-06-14,1.0,1.0,1.0
2012-06-15,2.0,2.0,2.0
2012-06-16,2.0,2.0,2.0
2012-06-17,2.0,2.0,2.0
2012-06-18,2.0,2.0,2.0


再看一个简单的例子，假设data1缺失了data2中存在的某个时间序列：

In [51]:
data2 = DataFrame(np.ones((6, 4), dtype=float) * 2,
                  columns=['a', 'b', 'c', 'd'],
                  index=pd.date_range('6/13/2012', periods=6))

In [52]:
spliced = pd.concat([data1.loc[:'2012-06-14'], data2.loc['2012-06-15':]])

In [53]:
spliced

,a,b,c,d
2012-06-12,1.0,1.0,1.0,NaN
2012-06-13,1.0,1.0,1.0,NaN
2012-06-14,1.0,1.0,1.0,NaN
2012-06-15,2.0,2.0,2.0,2.0
2012-06-16,2.0,2.0,2.0,2.0
2012-06-17,2.0,2.0,2.0,2.0
2012-06-18,2.0,2.0,2.0,2.0


combine_first可以引入合并点之前的数据，这样也就扩展了'd'项的历史：

In [54]:
spliced_filled = spliced.combine_first(data2)

In [55]:
spliced_filled

,a,b,c,d
2012-06-12,1.0,1.0,1.0,NaN
2012-06-13,1.0,1.0,1.0,2.0
2012-06-14,1.0,1.0,1.0,2.0
2012-06-15,2.0,2.0,2.0,2.0
2012-06-16,2.0,2.0,2.0,2.0
2012-06-17,2.0,2.0,2.0,2.0
2012-06-18,2.0,2.0,2.0,2.0


由于data2没有关于2012-06-12的数据，所以也就没有值被填充到那一天。

DataFrame也有一个类似的方法update，它可以实现就地更新。如果只想填充空洞，则必须传入overwrite=False才行：

In [56]:
spliced.update(data2, overwrite=False)

In [57]:
spliced

,a,b,c,d
2012-06-12,1.0,1.0,1.0,NaN
2012-06-13,1.0,1.0,1.0,2.0
2012-06-14,1.0,1.0,1.0,2.0
2012-06-15,2.0,2.0,2.0,2.0
2012-06-16,2.0,2.0,2.0,2.0
2012-06-17,2.0,2.0,2.0,2.0
2012-06-18,2.0,2.0,2.0,2.0


上面所讲的这些技术都可实现将数据中的符号替换为实际数据，但有时利用DataFrame的索引机制直接对列进行设置会更简单一些：

In [58]:
cp_spliced = spliced.copy()

In [59]:
cp_spliced[['a', 'c']] = data1[['a', 'c']]

In [60]:
cp_spliced

,a,b,c,d
2012-06-12,1.0,1.0,1.0,NaN
2012-06-13,1.0,1.0,1.0,2.0
2012-06-14,1.0,1.0,1.0,2.0
2012-06-15,1.0,2.0,1.0,2.0
2012-06-16,1.0,2.0,1.0,2.0
2012-06-17,1.0,2.0,1.0,2.0
2012-06-18,NaN,2.0,NaN,2.0


### 收益指数和累计收益
在金融领域中，收益（return）通常指的是某资产价格的百分比变化。

## 分组变换和分析
下面以一组假想的股票投资组合为例。首先随机生成1000个股票代码：

In [61]:
import random; random.seed(0)
import string

In [62]:
N = 1000
def rands(n):
    choices = string.ascii_uppercase
    return ''.join([random.choice(choices) for _ in range(n)])
tickers = np.array([rands(5) for _ in range(N)])

然后创建一个含有3列的DataFrame来承载这些假想数据，不过只选择部分股票组成该投资组合：

In [63]:
M = 500
df = DataFrame({'Momentum': np.random.randn(M) / 200 + 0.3,
                'Value': np.random.randn(M) / 200 + 0.08,
                'ShortInterest': np.random.randn(M) / 200 - 0.02},
               index=tickers[:M])

接下来，为这些股票随机创建一个行业分类。为了简单起见，我只选用了两个行业，并将映射关系保存在Series中：

In [66]:
ind_names = np.array(['FINANCIAL', 'TECH'])
sampler = np.random.randint(0, len(ind_names), N)
industries = Series(ind_names[sampler], index=tickers,
                    name='industry')

现在，我们就可以根据行业分类进行分组并执行分组聚合和变换了：

In [68]:
by_industry = df.groupby(industries)

In [69]:
by_industry.mean()

,Momentum,ShortInterest,Value
industry,,,
FINANCIAL,0.300100,-0.020163,0.080487
TECH,0.300204,-0.020338,0.079469


In [71]:
by_industry.describe().stack()

Momentum  ShortInterest       Value
industry                                              
FINANCIAL count  247.000000     247.000000  247.000000
          mean     0.300100      -0.020163    0.080487
          std      0.004804       0.005336    0.005041
          min      0.281550      -0.034198    0.065607
          25%      0.297670      -0.024243    0.077319
          50%      0.300456      -0.020284    0.080806
          75%      0.303128      -0.016051    0.083724
          max      0.312171      -0.004232    0.092300
TECH      count  253.000000     253.000000  253.000000
          mean     0.300204      -0.020338    0.079469
          std      0.005684       0.005287    0.004625
          min      0.284215      -0.034924    0.066362
          25%      0.296376      -0.023877    0.076444
          50%      0.300435      -0.020206    0.079370
          75%      0.304054      -0.016802    0.082523
          max      0.315630      -0.008270    0.091415

要对这些按行业分组的投资组合进行各种变换，我们可以编写自定义的变换函数。例如，行业内标准化处理，它广泛用于股票投资组合的构建过程：

In [73]:
# 行业内标准化处理
def zscore(group):
    return (group - group.mean()) / group.std()
df_stand = by_industry.apply(zscore)

这样处理之后，各行业的平均值为0，标准差为1：

In [74]:
df_stand.groupby(industries).agg(['mean', 'std'])

Momentum      ShortInterest              Value     
                   mean  std          mean  std          mean  std
industry                                                          
FINANCIAL -3.084083e-14  1.0 -1.241023e-15  1.0  1.542626e-15  1.0
TECH      -3.517915e-14  1.0  1.606532e-15  1.0 -5.002805e-15  1.0

内置变换函数（如rank）的用法会更简洁一些：

In [75]:
# 行业内降序排名
ind_rank = by_industry.rank(ascending=False)

In [76]:
ind_rank.groupby(industries).agg(['min', 'max'])

Momentum        ShortInterest        Value       
               min    max           min    max   min    max
industry                                                   
FINANCIAL      1.0  247.0           1.0  247.0   1.0  247.0
TECH           1.0  253.0           1.0  253.0   1.0  253.0

在股票投资组合的定量分析中，“排名和标准化”是一种很常见的变换运算组合。通过rank和zscore连接在一起即可完成整个变换过程，就像下面这样：

In [77]:
# 行业内排名和标准化
by_industry.apply(lambda x: zscore(x.rank()))

,Momentum,ShortInterest,Value
MYNBI,-1.413636,1.721557,-0.783798
QPMZJ,-0.895770,-0.363906,0.531863
PLSGQ,1.598819,-0.888233,1.557824
EJEYD,1.544159,-0.109321,-1.147870
TZIRW,1.612484,-0.204977,-1.667145
ZTEJD,0.464614,-1.571489,1.038549
XCVKP,1.343654,0.951755,0.461881
RDLNK,-1.021737,0.979748,0.139964
TUGRP,-0.405896,-0.727813,0.321917
OQIBZ,-1.407508,0.382623,-0.997554


### 分组因子暴露
因子分析（factor analysis）是投资组合定量管理中的一种技术。投资组合的特有量和性能（收益与损失）可以被分解为一个或多个表示投资组合权重的因子（风险因子就是其中之一）。例如，某只股票的价格与某个基准（比如标准普尔500指数）的械动性被称作其贝塔风险系数（beta，一种常见的风险因子）。

In [78]:
fac1, fac2, fac3 = np.random.rand(3, 1000)

In [85]:
ticker_subset = tickers.take(np.random.permutation(N)[:1000])

In [86]:
# 因子加权以及噪声
port = Series(0.7 * fac1 - 1.2 * fac2 + 0.3 * fac3 + np.random.rand(1000),
              index=ticker_subset)
factors = DataFrame({'f1': fac1, 'f2': fac2, 'f3': fac3},
                    index=ticker_subset)

各因子与投资组合之间的矢量相关性可能说明不了什么问题：

In [87]:
factors.corrwith(port)

f1    0.398422
f2   -0.716078
f3    0.210147
dtype: float64

### 十分位和四分位分析